In [14]:
from rl_agent import AgentGNN
import gym_zx
import gym
import torch
from torch_geometric.data import Batch, Data

In [20]:
SrH_env = gym.make("zx-v0", qubits=6, depth=None, env_id=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rl_agent = AgentGNN(None,device=device).to(device)
rl_agent.load_state_dict(
        torch.load("/home/jordi.riu/Copt-cquere/rl-zx/state_dict_0model5x70_twoqubits_new.pt", map_location=torch.device("cpu"))
    )  
rl_agent.eval()


AgentGNN(
  (global_attention_critic): GlobalAttention(gate_nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  ), nn=Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (critic_gnn): Sequential(
    (0): GATv2Conv(12, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
    (8): GATv2Conv(32, 32, heads=1)
    (9): ReLU()
  )
  (actor_gnn): Sequential(
    (0): GATv2Conv(17, 32, heads=1)
    (1): ReLU()
    (2): GATv2Conv(32, 32, heads=1)
    (3): ReLU()
    (4): GATv2Conv(32, 32, heads=1)
    (5): ReLU()
    (6): GATv2Conv(32, 32, heads=1)
    (7): ReLU()
    (8): G

In [31]:
obs0, reset_info = SrH_env.reset()
policy_items, value_items = reset_info["graph_obs"]
value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

In [32]:

done = False
while not done:
    
    action, _, _, value, logits, action_ids = rl_agent.get_action_and_value(next_obs_graph, device=device)
    next_obs, reward, done, deprecated, info = SrH_env.step(action_ids.cpu().numpy())
    policy_items, value_items = info["graph_obs"]
    value_graph = [Data(x=value_items[0], edge_index=value_items[1], edge_attr=value_items[2])]
    policy_graph = [Data(x=policy_items[0], edge_index=policy_items[1], edge_attr=policy_items[2], y=policy_items[3])]
    next_obs_graph = (Batch.from_data_list(policy_graph), Batch.from_data_list(value_graph))

Win vs Pyzx:  1  Episode Gates:  74 Single gates: 133 Episode Len 264 Opt Episode Len 227


In [18]:
SrH_env.pyzx_data

{'qubits': 6,
 'gates': 150,
 'tcount': 24,
 'clifford': 47,
 'CNOT': 57,
 'CZ': 22,
 'had': 16,
 'twoqubits': 79}

In [33]:
import pyzx as zx
graph = SrH_env.final_circuit.to_graph()
zx.to_graph_like(graph)
zx.flow_2Q_simp(graph)
circuit = zx.extract_simple(graph).to_basic_gates()

circuit = zx.basic_optimization(circuit).to_basic_gates()
SrH_env.final_circuit.stats()


'Circuit  on 10 qubits with 133 gates.\n        24 is the T-count\n        109 Cliffords among which\n        74 2-qubit gates (59 CNOT, 15 other) and\n        12 Hadamard gates.'

In [34]:
circuit.stats()

'Circuit  on 10 qubits with 114 gates.\n        24 is the T-count\n        90 Cliffords among which\n        63 2-qubit gates (51 CNOT, 12 other) and\n        8 Hadamard gates.'

In [13]:
graph = circuit.to_graph()
zx.to_graph_like(graph)
zx.flow_2Q_simp(graph)
circuit = zx.extract_simple(graph).to_basic_gates()

circuit = zx.basic_optimization(circuit).to_basic_gates()
circuit.stats()

'Circuit  on 10 qubits with 147 gates.\n        24 is the T-count\n        123 Cliffords among which\n        76 2-qubit gates (60 CNOT, 16 other) and\n        15 Hadamard gates.'